In [16]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
POSTGRES_PASSWORD = os.environ['POSTGRES_PASSWORD']

In [2]:
nba = pd.read_csv('nba.csv', low_memory = False)

In [3]:
nba.head(3).T

,0,1,2
Unnamed: 0,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
game_date,2022-02-17,2022-02-17,2022-02-17
OT,0,0,0
H_A,A,A,A
...,...,...,...
SG%,36.0,0.0,32.0
SF%,60.0,4.0,67.0
PF%,4.0,85.0,0.0
C%,0.0,11.0,0.0


In [4]:
nba = nba.drop(['Inactives'], axis = 1)
nba.head(3).T

,0,1,2
Unnamed: 0,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
game_date,2022-02-17,2022-02-17,2022-02-17
OT,0,0,0
H_A,A,A,A
...,...,...,...
SG%,36.0,0.0,32.0
SF%,60.0,4.0,67.0
PF%,4.0,85.0,0.0
C%,0.0,11.0,0.0


In [5]:
players = nba[["player_id", "player"]].drop_duplicates()
players

,player_id,player
0,kispeco01,Corey Kispert
1,kuzmaky01,Kyle Kuzma
2,caldwke01,Kentavious Caldwell-Pope
3,netora01,Raul Neto
4,bryanth01,Thomas Bryant
...,...,...
109702,frazimi01,Michael Frazier
110441,howarwi01,William Howard
110913,mbahalu01,Luc Mbah a Moute
111399,bowmaky01,Ky Bowman


In [6]:
nba = nba.drop(['player'], axis = 1)

In [7]:
games = nba[['game_id', "game_date", "OT", 'season']].drop_duplicates()
nba = nba.drop(['game_date', "OT", 'season'], axis=1)

In [8]:
nba.head(3).T

,0,1,2
Unnamed: 0,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
H_A,A,A,A
Team_Abbrev,WAS,WAS,WAS
Team_Score,117,117,117
...,...,...,...
SG%,36.0,0.0,32.0
SF%,60.0,4.0,67.0
PF%,4.0,85.0,0.0
C%,0.0,11.0,0.0


# Third Normal Form

In [9]:
nba.columns

Index(['Unnamed: 0', 'game_id', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev', 'Opponent_Score',
       'Opponent_pace', 'Opponent_efg_pct', 'Opponent_tov_pct',
       'Opponent_orb_pct', 'Opponent_ft_rate', 'Opponent_off_rtg', 'player_id',
       'starter', 'mp', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft',
       'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'plus_minus', 'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct',
       'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct',
       'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg',
       'def_rtg', 'bpm', 'minutes', 'double_double', 'triple_double', 'DKP',
       'FDP', 'SDP', 'DKP_per_minute', 'FDP_per_minute', 'SDP_per_minute',
       'pf_per_minute', 'ts', 'last_60_minutes_per_game_starting',
       'last_60_minutes_per_game_be

In [30]:
team_game = nba[['game_id', 'Team_Abbrev', 'H_A', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev']].drop_duplicates()

In [11]:
player_game = nba.drop(['Unnamed: 0', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev', 'Opponent_Score',
       'Opponent_pace', 'Opponent_efg_pct', 'Opponent_tov_pct',
       'Opponent_orb_pct', 'Opponent_ft_rate', 'Opponent_off_rtg'], axis = 1)

In [12]:
player_game

,game_id,player_id,starter,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,202202170BRK,kispeco01,1,32:30,6,9,0.667,4,6,0.667,...,0.061538,9.00,31.716667,22.017778,1.0,36.0,60.0,4.0,0.0,46.253586
1,202202170BRK,kuzmaky01,1,30:16,2,7,0.286,0,3,0.000,...,0.099119,7.44,34.324000,18.475954,0.0,0.0,4.0,85.0,11.0,52.152590
2,202202170BRK,caldwke01,1,25:26,3,7,0.429,1,3,0.333,...,0.000000,7.00,29.820290,16.051693,0.0,32.0,67.0,0.0,0.0,47.021807
3,202202170BRK,netora01,1,20:40,5,7,0.714,1,1,1.000,...,0.048387,7.88,29.920833,14.603922,90.0,10.0,0.0,0.0,0.0,27.603314
4,202202170BRK,bryanth01,1,14:04,5,6,0.833,0,1,0.000,...,0.000000,6.88,20.095833,14.538095,0.0,0.0,0.0,0.0,100.0,36.472537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112118,202003070GSW,wiggian01,1,37:04:00,3,10,0.300,0,0,0.000,...,0.107914,13.08,33.110667,19.232562,0.0,2.0,77.0,21.0,0.0,57.207786
112119,202003070GSW,toscaju01,1,27:43:00,3,6,0.500,0,2,0.000,...,0.036079,6.00,25.470833,20.228571,5.0,45.0,43.0,7.0,0.0,58.202391
112120,202003070GSW,bendedr01,0,13:15,4,4,1.000,2,2,1.000,...,0.150943,4.00,24.083333,13.228788,0.0,0.0,0.0,9.0,91.0,49.630640
112121,202003070GSW,muldemy01,1,31:48:00,5,10,0.500,3,7,0.429,...,0.094340,12.64,34.783333,27.691667,0.0,44.0,48.0,8.0,0.0,58.923515


## SQL

In [18]:
dbserver = psycopg2.connect(
    user='postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = '5432'
)
#Host needs to be set equal to whatever the service is named in your Docker Compose file
dbserver.autocommit=True

In [19]:
cursor = dbserver.cursor()

In [21]:
try:
    cursor.execute('CREATE DATABASE nba')
except:
    cursor.execute("DROP DATABASE nba")
    cursor.execute('CREATE DATABASE nba')

In [25]:
engine = create_engine("postgresql+psycopg2://{user}:{pw}@{service}/{db}".format(
    user = 'postgres', 
    pw = POSTGRES_PASSWORD,
    service = 'postgres',
    db = 'nba'
))

In [43]:
games.columns = [x.lower() for x in games.columns]
players.columns = [x.lower() for x in players.columns]
team_game.columns = [x.lower() for x in team_game.columns]
player_game.columns = [x.lower() for x in player_game.columns]
games.to_sql('games', con = engine, index = False, chunksize = 1000, if_exists = 'replace')

3197

In [44]:
players.to_sql('players', con = engine, index = False, chunksize = 1000, if_exists = 'replace')

812

In [45]:
team_game.to_sql('team_game', con = engine, index = False, chunksize = 1000, if_exists = 'replace')

6394

In [50]:
player_game.to_sql('player_game', con = engine, index = False, chunksize = 500, if_exists = 'replace')

112123

In [51]:
myquery = '''
SELECT * 
FROM games
'''
pd.read_sql_query(myquery, con = engine)

,game_id,game_date,ot,season
0,202202170BRK,2022-02-17,0,2022
1,202202170CHO,2022-02-17,2,2022
2,202202170LAC,2022-02-17,0,2022
3,202202170MIL,2022-02-17,0,2022
4,202202170NOP,2022-02-17,0,2022
...,...,...,...,...
3192,202001080GSW,2020-01-08,0,2020
3193,202008020HOU,2020-08-02,0,2020
3194,201911060HOU,2019-11-06,0,2020
3195,201912250GSW,2019-12-25,0,2020


In [52]:
myquery = '''
SELECT * 
FROM games
WHERE game_date = '2021-04-12'
'''
pd.read_sql_query(myquery, con = engine)

,game_id,game_date,ot,season
0,202104120GSW,2021-04-12,0,2021
1,202104120NYK,2021-04-12,0,2021
2,202104120ORL,2021-04-12,0,2021
3,202104120UTA,2021-04-12,0,2021
4,202104120DAL,2021-04-12,0,2021
5,202104120NOP,2021-04-12,0,2021
6,202104120MEM,2021-04-12,0,2021
7,202104120PHO,2021-04-12,0,2021


In [56]:
myquery = '''
SELECT pg.game_id, pg.player_id, pg.pts, tg.team_score,
    CAST(pg.pts AS float)/CAST(tg.team_score AS float) AS point_percent
FROM player_game pg
INNER JOIN team_game tg
    ON pg.game_id = tg.game_id
WHERE pg.pts>40 AND tg.team_score>120
'''
pd.read_sql_query(myquery, con = engine)

,game_id,player_id,pts,team_score,point_percent
0,202202170MIL,embiijo01,42,123,0.341463
1,202202170NOP,doncilu01,49,125,0.392000
2,202110250CHO,tatumja01,41,129,0.317829
3,202110250CHO,tatumja01,41,140,0.292857
4,202110200NYK,brownja02,46,138,0.333333
...,...,...,...,...,...
389,202007310DAL,hardeja01,49,153,0.320261
390,202007310DAL,hardeja01,49,149,0.328859
391,202008090SAC,riverau01,41,129,0.317829
392,202008090SAC,riverau01,41,129,0.317829
